In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import scipy.integrate as spi


In [ ]:
# 参数设置
S0 = 1      # 初始资产价格
T = 35.0    # 总时长（例如1年）
r = 0.06    # 风险无关利率
sigma = 0.2 # 波动率
dt = 0.01   # 时间步长
N = 100     # 总步数
g = 0
lamda = 1/35

In [ ]:
# 定义股价模拟函数
def stock_SDE(S0, r, sigma, dt, N, l):
    S = np.zeros(N)
    S[0] = S0
    for t in range(1, N):
        Z = np.random.normal(0, 1)
        S[t] = S[t-1] * np.exp((r - l - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z)
    return S[N-1]

# 定义期权收益函数
def option_payoff(g, t, S0, r, sigma, dt, N, l):
    return max(np.exp(g * t) - stock_SDE(S0, r, sigma, dt, N, l), 0)


# 定义单次积分函数
def single_V(t, l):
    return lamda * np.exp(-lamda * t) * np.exp(-r * t) * option_payoff(g, t, S0, r, sigma, dt, N, l)

# 定义费用流函数
def fee_flows(l):
    return l / (lamda + l)

In [ ]:
# 创建神经网络模型
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(1, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = Net()

# 定义损失函数和优化器
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.00001)

# 训练模型
M = 100  # 进行多次积分以估算期望

for epoch in range(1000):
    optimizer.zero_grad()
    
    # 随机生成一个 l 值
    l_candidate = np.random.uniform(0, 0.001)
    
    # 估算多次积分的均值
    results = []
    for _ in range(M):
        result, _ = spi.quad(single_V, 0, T, args=(l_candidate,))
        results.append(result)
    
    average_payoff = np.mean(results)
    
    # 使用神经网络来估计 l
    l_pred = model(torch.tensor([[l_candidate]], dtype=torch.float32))
    
    # 计算损失
    loss = criterion(l_pred, torch.tensor([[average_payoff]], dtype=torch.float32))
    
    # 反向传播和优化
    loss.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')

# 使用训练后的模型来估计最优的 l
with torch.no_grad():
    l_pred = model(torch.tensor([[0.5]], dtype=torch.float32))
    optimal_l = l_pred.item()

print("Optimal l:", optimal_l)